# Aspect term extraction and sentiment classification

This Notebook is created and Run in colab by Devansh Mistry. Deployed on Gradio.

This Notebook is using [PyABSA](https://https://github.com/yangheng95/PyABSA) library, Citation

In [ ]:
@article{YangL22,
  author    = {Heng Yang and
               Ke Li},
  title     = {PyABSA: Open Framework for Aspect-based Sentiment Analysis},
  journal   = {CoRR},
  volume    = {abs/2208.01368},
  year      = {2022},
  url       = {https://doi.org/10.48550/arXiv.2208.01368},
  doi       = {10.48550/arXiv.2208.01368},
  eprinttype = {arXiv},
  eprint    = {2208.01368},
  timestamp = {Tue, 08 Nov 2022 21:46:32 +0100},
  biburl    = {https://dblp.org/rec/journals/corr/abs-2208-01368.bib},
  bibsource = {dblp computer science bibliography, https://dblp.org}
}

In [ ]:
! pip install pyabsa==1.16.27 

In [ ]:
from pyabsa import available_checkpoints
checkpoint_map = available_checkpoints('atepc', show_ckpts=True)

In [ ]:
from pyabsa import ABSADatasetList
from pyabsa import ATEPCCheckpointManager

aspect_extractor = ATEPCCheckpointManager.get_aspect_extractor(checkpoint='english',
                                   auto_device=False  # False means load model on CPU
                                   )

#Inference

In [ ]:
#indvidual
examples = ['But the staff was so perfect to us, but the service was bad .',
            ]
inference_source = examples
atepc_result = aspect_extractor.extract_aspect(inference_source=inference_source,  #
                          save_result=True,
                          print_result=True,  # print the result
                          pred_sentiment=True,  # Predict the sentiment of extracted aspect terms
                          )

In [ ]:
inference_source = ABSADatasetList.SemEval
atepc_result = aspect_extractor.extract_aspect(inference_source=inference_source,  #
                          save_result=True,
                          print_result=True,  # print the result
                          pred_sentiment=True,  # Predict the sentiment of extracted aspect terms
                          )

# Traning on DATA

In [ ]:
import os

import findfile

from pyabsa.functional import ATEPCModelList
from pyabsa.functional import Trainer, ATEPCTrainer
from pyabsa.functional import ABSADatasetList
from pyabsa.functional import ATEPCConfigManager
from pyabsa.functional.dataset import DatasetItem

atepc_config = ATEPCConfigManager.get_atepc_config_english()

atepc_config.pretrained_bert = 'yangheng/deberta-v3-base-absa-v1.1'
atepc_config.model = ATEPCModelList.FAST_LCF_ATEPC
dataset_path = ABSADatasetList.Restaurant14
# dataset_path = ' local dataset path'

aspect_extractor = ATEPCTrainer(config=atepc_config,
                  dataset=dataset_path,
                  from_checkpoint='',  # set checkpoint to train on the checkpoint.
                  checkpoint_save_mode=1,
                  auto_device=True
                  ).load_trained_model()



---

# Deployment

In [ ]:
! pip install gradio

In [ ]:
import gradio as gr
import pandas as pd

In [ ]:
def inference(text):
    result = aspect_extractor.extract_aspect(inference_source=[text],
                                             pred_sentiment=True)

    result = pd.DataFrame({
        'aspect': result[0]['aspect'],
        'sentiment': result[0]['sentiment']
    })

    return result

In [ ]:
if __name__ == '__main__':
    iface = gr.Interface(
        fn=inference,
        inputs=["text"],
        examples=[['The wine list is incredible and extensive and diverse , the food is all incredible and the staff was all very nice ,'
                   ' good at their jobs and cultured .'],
                  ['Though the menu includes some unorthodox offerings (a peanut butter roll, for instance), the classics are pure and '
                   'great--we have never had better sushi anywhere, including Japan.'],
                  ['Everything, from the soft bread, soggy salad, and 50 minute wait time, with an incredibly rude service to deliver'
                   ' below average food .'],
                  ['Even though it is running Snow Leopard, 2.4 GHz C2D is a bit of an antiquated CPU and thus the occasional spinning '
                   'wheel would appear when running Office Mac applications such as Word or Excel .'],
                  ['This demo is trained on the laptop and restaurant and other review datasets from ABSADatasets (https://github.com/yangheng95/ABSADatasets)'],
                  ['To fit on your data, please train the model on your own data, see the PyABSA (https://github.com/yangheng95/PyABSA)'],
                  ],
        outputs="dataframe",
        title='Aspect Term Extraction for Short Texts (powered by PyABSA)'
    )

    iface.launch()